In this code, I have used zero-shot classifier from bert , huggingface, in order to label the dimension of each comment.




**steps**:
1- assiging one of the ten dimensions to each comment .

**warninI did not assign label for posts by this code. since I assume that according to the first method, most post are based on the knwoledge, information, or fact.

In [5]:
import tensorflow as tf
# Check if a GPU is available.
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

GPU is available


In [6]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [7]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [8]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("TensorFlow version: ", tf.__version__)


Num GPUs Available:  1
TensorFlow version:  2.17.1


In [12]:
#!pip install transformers
#!pip install gdown
#!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=2ddf1c700b0ebd6f354ee0c6620891316f297cfab35930b13544dfe131c5f0b9
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [13]:
import time

# A small computation to test GPU usage
with tf.device('/GPU:0'):
    start = time.time()
    a = tf.random.normal([10000, 10000])
    b = tf.random.normal([10000, 10000])
    c = tf.matmul(a, b)
    print("Computation done in: ", time.time() - start, "seconds")


Computation done in:  0.11672306060791016 seconds


In [14]:
# tenserflwo only uses gpu

import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
        print("GPU memory configured.")
    except RuntimeError as e:
        print(e)


Physical devices cannot be modified after being initialized


# **Start from this part: **

In [16]:

import pandas as pd





# datase merged_table :
# !gdown "https://drive.google.com/uc?id=1ueYnNJGl4CT5jgo1IDI5Gs4hjwhU-hI-"

# dataset= posts_with_dimesions
#https://drive.google.com/file/d/1X9VaJGzO8-ECO7YSu2FDu4GZwCIKLWto/view?usp=sharing


#!gdown "https://drive.google.com/uc?id=1X9VaJGzO8-ECO7YSu2FDu4GZwCIKLWto"


#url for 6000= https://drive.google.com/file/d/1PS8MS_0DVu6Qj3eozRr8Hkmj-3KC4T23/view?usp=drive_link
# URL for ten dimensions = https://drive.google.com/file/d/1aBqtSnWow0RGkkxgzr8bf2l-mOS8CiFw/view?usp=drive_link
#!gdown "https://drive.google.com/uc?id=1PS8MS_0DVu6Qj3eozRr8Hkmj-3KC4T23" # dataset of summery2
!gdown "https://drive.google.com/uc?id=1aBqtSnWow0RGkkxgzr8bf2l-mOS8CiFw"    # dataset often dimensions

print("\n=== Loading Dataset ===")
#data = pd.read_csv('tendims_final2.csv', encoding='ISO-8859-1')

data = pd.read_csv('tendims_final2.csv')


Downloading...
From (original): https://drive.google.com/uc?id=1aBqtSnWow0RGkkxgzr8bf2l-mOS8CiFw
From (redirected): https://drive.google.com/uc?id=1aBqtSnWow0RGkkxgzr8bf2l-mOS8CiFw&confirm=t&uuid=34c879e8-cf9c-418e-8468-56d2f3af7123
To: /content/tendims_final2.csv
100% 504M/504M [00:06<00:00, 77.8MB/s]

=== Loading Dataset ===


In [17]:
#!ls

In [18]:
data.head()

,level_0,index,post_id,link_id,subreddit,parent_id,redditor_id,created_at,body,score,...,identity_mean_text,identity_max_text,identity_min_text,identity_std_text,romance_mean_text,romance_max_text,romance_min_text,romance_std_text,dimension,dimension_text
0,0,23894,j4m5rux,10djn5r,changemyview,t1_j4m5hn6,n1yr8,2023-01-16 19:20:40+00:00,"Yes, Sub-Saharan Africa is the area that lies ...",{'2024-10-29T13:36:05': 4},...,0.639293,0.766554,0.53668,0.06238,0.038944,0.690232,0.000015,0.157973,identity,knowledge
1,1,24022,j4m8y3k,10djn5r,changemyview,t1_j4m3epz,mp308,2023-01-16 19:40:10+00:00,But then how do we know there isn't just one r...,{'2024-10-29T13:36:06': 7},...,0.639293,0.766554,0.53668,0.06238,0.038944,0.690232,0.000015,0.157973,knowledge,knowledge
2,2,24396,j4mjpel,10djn5r,changemyview,t1_j4mjj36,suspended:UBC_Guy_,2023-01-16 20:47:08+00:00,Yeah I think around the Bengal area is where s...,{'2024-10-29T13:36:39': 1},...,0.639293,0.766554,0.53668,0.06238,0.038944,0.690232,0.000015,0.157973,knowledge,knowledge
3,3,24973,j4s5e22,10djn5r,changemyview,t1_j4s48zs,suspended:UBC_Guy_,2023-01-17 22:53:21+00:00,Just because it’s not utilitarian or useful in...,{'2024-10-29T13:36:15': 1},...,0.639293,0.766554,0.53668,0.06238,0.038944,0.690232,0.000015,0.157973,conflict,knowledge
4,4,25019,j4nv1wp,10djn5r,changemyview,t1_j4nutkf,suspended:UBC_Guy_,2023-01-17 01:59:23+00:00,I was hoping you’d think it obvious that ethni...,{'2024-10-29T13:36:16': 1},...,0.639293,0.766554,0.53668,0.06238,0.038944,0.690232,0.000015,0.157973,knowledge,knowledge


In [19]:
data['body'][3]

'Just because it’s not utilitarian or useful in showing trait differences between groups doesn’t mean it’s not true. Large-scale Ancestral groupings are real amd provable.'

In [20]:
import torch
import transformers
print(torch.__version__)
print(transformers.__version__)


2.5.1+cu121
4.47.1


In [22]:
from transformers import pipeline

# Load Zero-Shot Classification pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Define your target dimensions
dimensions = ["knowledge", "trust", "similarity", "power", "status", "support", "romance", "identity","fun","conflict"]
#dimensions = ["knowledge", "trust", "similarity"]
#dimensions_3 = ["knowledge"]

Device set to use cuda:0


In [23]:
def assign_dimension_with_bert(comment):
    """
    Use BERT-based zero-shot classification to assign a dimension.
    """
    if not isinstance(comment, str) or comment.strip() == "":
        return "Unknown"  # Handle empty or invalid comments

    result = classifier(comment, dimensions)  # Classify the comment
    print("rrrrrrrrrrrrrrrrrrrr",result)
    predicted_dimension = result['labels'][0]  # Get the top predicted dimension
    print("pppppppppppppppppp",predicted_dimension)
    return predicted_dimension


In [ ]:
#data1= data[1:2000]

In [ ]:
#data1.head()

,post_id,link_id,subreddit,parent_id,redditor_id,created_at,body,score,edited,removed,...,knowledge_min,knowledge_std,similarity_mean,similarity_max,similarity_min,similarity_std,trust_mean,trust_max,trust_min,trust_std
1,k3anc3z,16yqq5w,changemyview,t3_16yqq5w,deleted,2023-10-03 17:34:08+00:00,I'll share an anecdote. I'm well read enough t...,{'2024-10-28T14:52:26': 12},True,NaN,...,0.390996,0.213204,0.348924,0.681144,0.004553,0.233644,0.136963,0.545794,5.712525e-07,0.200768
2,k3aek0j,16yqq5w,changemyview,t3_16yqq5w,5kww6,2023-10-03 16:40:00+00:00,Please forgive not having links. I wrote this ...,{'2024-10-28T14:52:26': 23},False,NaN,...,0.275995,0.192097,0.381131,0.996932,0.003916,0.352987,0.187493,0.816639,5.709421e-07,0.270602
3,k3a4ekd,16yqq5w,changemyview,t3_16yqq5w,4bnx7,2023-10-03 15:32:05+00:00,You’re not quite thinking about this the right...,{'2024-10-28T14:52:26': 67},True,NaN,...,0.453278,0.133724,0.289383,0.995853,0.006129,0.281469,0.156220,0.907190,5.714884e-07,0.292195
4,k39xcxh,16yqq5w,changemyview,t3_16yqq5w,suspended:barbodelli,2023-10-03 14:38:43+00:00,[https://cei.org/blog/wrong-again-50-years-of-...,{'2024-10-28T14:52:27': 235},False,NaN,...,0.939058,0.000000,0.005346,0.005346,0.005346,0.000000,0.548194,0.548194,5.481943e-01,0.000000
5,k3a4li3,16yqq5w,changemyview,t3_16yqq5w,uh386cg2,2023-10-03 15:33:30+00:00,Your view is a goalpost shift. And kind of sil...,{'2024-10-28T14:52:27': 33},False,NaN,...,0.475956,0.137652,0.421186,0.596667,0.146807,0.121111,0.251743,0.726590,5.731871e-07,0.223485


In [24]:
# Apply the function to the 'body' column
data['semantic_dimension'] = data['cleaned_body'].apply(assign_dimension_with_bert)

# Display the updated dataset
print(data[['cleaned_body', 'semantic_dimension']].head())

rrrrrrrrrrrrrrrrrrrr {'sequence': "yes, sub-saharan africa is the area that lies south of the sahara. you're missing north africa.", 'labels': ['similarity', 'knowledge', 'status', 'support', 'identity', 'power', 'trust', 'conflict', 'fun', 'romance'], 'scores': [0.2454807609319687, 0.2010679692029953, 0.16395239531993866, 0.0985422283411026, 0.07552512735128403, 0.05825914070010185, 0.05386033281683922, 0.049272555857896805, 0.040124036371707916, 0.013915487565100193]}
pppppppppppppppppp similarity
rrrrrrrrrrrrrrrrrrrr {'sequence': "but then how do we know there isn't just one race with sub-populations within it?", 'labels': ['conflict', 'knowledge', 'status', 'power', 'identity', 'support', 'similarity', 'fun', 'trust', 'romance'], 'scores': [0.2609199583530426, 0.17296482622623444, 0.13697947561740875, 0.12590625882148743, 0.08872471749782562, 0.08525597304105759, 0.07555563002824783, 0.03727300837635994, 0.009375112131237984, 0.007045008707791567]}
pppppppppppppppppp conflict
rrrrr

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Streaming output truncated to the last 5000 lines.
rrrrrrrrrrrrrrrrrrrr {'sequence': 'terrific vanish distinct water yoke dinosaurs important offbeat serious spectacular -- mass edited with redact.dev', 'labels': ['status', 'fun', 'power', 'support', 'conflict', 'similarity', 'identity', 'knowledge', 'trust', 'romance'], 'scores': [0.28017327189445496, 0.2341744601726532, 0.19347262382507324, 0.08347716182470322, 0.06494026631116867, 0.06352366507053375, 0.0329228937625885, 0.02478715591132641, 0.014498274773359299, 0.008030230179429054]}
pppppppppppppppppp status
rrrrrrrrrrrrrrrrrrrr {'sequence': "you're massively over estimating the damage and frequency of these storms. even if we got three or four category 5 storms a year, which we do not, that wouldn't be enough to make rebuilding not worth it. the majority of the state simply gets a lot of water and some property damage. only the landing port gets devastated, and it literally can be any part of the coastline that could get struck.

In [25]:
data['cleaned_body'][40]

'all europeans, tamils and semitic peoples to my knowledge are all caucasians. the origin of australian aboriginese is disputed because weve observed a phenotypic mixture between various african/asian traits i think the best theory is that they migrated from africa, came to asian along the way, and ended up there really long ago, and have since highly differentiated on their own. i just wouldnt classify them as a major race, because their population and culture has dwindled massively as a result of colonialism'

In [26]:
data['semantic_dimension'].value_counts()

,count
semantic_dimension,
conflict,17105
similarity,14716
status,4443
power,2732
support,2647
knowledge,2058
fun,1167
trust,331
identity,158


In [27]:
data['dimension'].value_counts()

,count
dimension,
knowledge,29171
conflict,8307
identity,3955
similarity,1069
power,922
trust,732
support,508
fun,437
status,206


In [ ]:
# testing one text based on zero shot classifier:

from transformers import pipeline

# Load Zero-Shot Classification pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Example text
text = "youre not quite thinking about this the right way. the united states is responsible for 15 of global emissions europe another 15. china is 30 the rest of world is another 40 and growing. the us and europe might be able to agree on things like the paris accords to try to cut their emissions by 50 but they havent been effective to date. even if the us and europe hit the paris goals it just translates to a cut of 15 globally. thats not enough according to more dire projections and worse yet the developing world is likely to increase its emissions by that much in the same time period. so if you want to hit 1020 year goals around point of no return you must address the developing world and there are no viable solutions proposed. liberals will simultaneously argue that we need to make rapid climate change progress while also suggesting the developing world will come to the table once they modernize and raise their standards of living. cool but doing so will cause tremendous amounts of emissions in the meantime and probably puts us on a 50100 year timeline before they turn the corner into reduction. effectively you can believe one of two things  a 50100 year timeline is ok to address climate change. thats about the amount of time we need for the developing world  banking on not yet realized technology breakthroughs.  climate change is more dire and we need much more progress in 1020 years. that necessitates major austerity from developed nations as well as a real answer to the developing world and its somewhat hard to arrive at any conclusion than a thanos like strategy of lowering global population fairly rapidly. the distinction between 1020 years being a point of no return for long term damage  risk of negative feedback loops vs a rapid apocalypse isnt the real issue here. the real issue is that liberal interpretations of data do not match their policy advocacy and a lot of yelling about a problem that feels unactionable will eventually turn into apathy."

# Labels
labels = ["Knowledge", "Trust", "Similarity"]

# Perform classification
result = classifier(text, labels)

# Output the results
print(result)


In [ ]:
# this partis proceed (again) in another script so ignore this part.


'''
# Define threshold for small differences between means
mean_diff_threshold = 0.15  # Updated threshold for closeness of mean values

def assign_dimension_with_low_std(row, mean_diff_threshold):
    # Step 1: Extract mean values for all dimensions
    mean_values = {
        dim: row[f'{dim}_mean']
        for dim in ['knowledge', 'trust', 'similarity']
        if not pd.isna(row[f'{dim}_mean'])
    }

    # Step 2: Determine the dominant dimension based on mean values
    if mean_values:
        sorted_means = sorted(mean_values.items(), key=lambda x: x[1], reverse=True)
        max_dim, max_mean = sorted_means[0]  # Dimension with the highest mean
        if len(sorted_means) > 1:
            second_max_dim, second_max_mean = sorted_means[1]  # Second highest mean

            # Check if the difference between top two means is small
            if max_mean - second_max_mean < mean_diff_threshold:
                # Use the dimension with the smallest std to break the tie
                dimensions_to_compare = [max_dim, second_max_dim]
                std_values = {
                    dim: row[f'{dim}_std']
                    for dim in dimensions_to_compare
                    if not pd.isna(row[f'{dim}_std'])
                }
                if std_values:
                    dominant_dimension = min(std_values, key=std_values.get)  # Dimension with the smallest std
                    return dominant_dimension
                else:
                    return 'Unknown'  # If std values are missing for both dimensions

        # If no ambiguity, assign based on the highest mean
        return max_dim

    # Step 3: If no mean values are available, fallback to the dimension with the smallest std
    std_values = {
        dim: row[f'{dim}_std']
        for dim in ['knowledge', 'trust', 'similarity']
        if not pd.isna(row[f'{dim}_std'])
    }
    if std_values:
        dominant_dimension = min(std_values, key=std_values.get)  # Dimension with the smallest std
        return dominant_dimension
    return 'Unknown'

# Apply the function to your dataset
data1['dominant_dimension_low_std'] = data.apply(
    assign_dimension_with_low_std, axis=1, mean_diff_threshold=mean_diff_threshold
)

# Display a sample of the updated DataFrame
data1[['body_clean', 'knowledge_mean', 'trust_mean', 'similarity_mean', 'dominant_dimension_low_std' , 'semantic_dimension']].head()
'''


In [ ]:
# this partis proceed (again) in another script so ignore this part.


'''

# Define thresholds
mean_diff_threshold = 0.1  # Threshold for determining if mean values are "close"

def assign_dimension(row, mean_diff_threshold=0.1):
    # Step 1: Extract mean and std values for all dimensions
    dimensions = ['knowledge', 'similarity', 'trust']
    mean_values = {dim: row[f'{dim}_mean'] for dim in dimensions if not pd.isna(row[f'{dim}_mean'])}
    std_values = {dim: row[f'{dim}_std'] for dim in dimensions if not pd.isna(row[f'{dim}_std'])}

    # Step 2: Check if mean values exist
    if mean_values:
        # Find the dimension with the highest mean and second-highest mean
        sorted_means = sorted(mean_values.items(), key=lambda x: x[1], reverse=True)
        max_dim, max_mean = sorted_means[0]  # Dimension with the highest mean
        if len(sorted_means) > 1:
            second_max_dim, second_max_mean = sorted_means[1]  # Dimension with the second-highest mean
            # Check if the means are "close"
            if abs(max_mean - second_max_mean) <= mean_diff_threshold:
                # Means are close; use std to determine the dominant dimension
                if std_values:
                    min_std_dim = min(std_values, key=std_values.get)  # Dimension with the lowest std
                    return min_std_dim
            # If means are not close, return the dimension with the highest mean
            return max_dim
        else:
            # If only one mean value exists, it's the dominant dimension
            return max_dim

    # Step 3: If no mean values are available, fallback to std
    if std_values:
        min_std_dim = min(std_values, key=std_values.get)  # Dimension with the lowest std
        return min_std_dim
    return 'Unknown'

# Apply the function to your dataset
data['dominant_dimension'] = data.apply(assign_dimension, axis=1, mean_diff_threshold=mean_diff_threshold)

# Display a sample of the updated DataFrame
data[['body_clean', 'knowledge_mean', 'similarity_mean', 'trust_mean', 'dominant_dimension']].head()

'''


In [ ]:
def assign_dimension_with_bert(comment):
    """
    Use BERT-based zero-shot classification to assign a dimension.
    """
    if not isinstance(comment, str) or comment.strip() == "":
        return "Unknown"  # Handle empty or invalid comments

    result = classifier(comment, dimensions)  # Classify the comment
    print("rrrrrrrrrrrrrrrrrrrr",result)
    predicted_dimension = result['labels'][0]  # Get the top predicted dimension
    print("pppppppppppppppppp",predicted_dimension)
    return predicted_dimension


In [ ]:
# Apply the function to the 'body' column
data['semantic_ten_dimension'] = data['cleaned_text'].apply(assign_dimension_with_bert)

# Display the updated dataset
print(data[['cleaned_text', 'semantic_dimension' , 'dimension']].head())

In [28]:
# Example: Save the DataFrame
data.to_csv('tendimension2_zeroshot.csv', index=False)
print("Dataset saved as 'tendimension2_zeroshot.csv'")


Dataset saved as 'tendimension2_zeroshot.csv'


In [29]:
from google.colab import files

# Download the dataset
files.download('tendimension2_zeroshot.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>